# 猫狗大战 毕业项目——Fine-tuning ResNet50


## 开始
导入一切并我们设置所使用的GPU。
- dev0: GTX1070Ti

In [1]:
%matplotlib inline
#import utilities
import os
import shutil
import numpy as np
import random
from tqdm import tqdm  
from time import time
from PIL import Image
import h5py
import pandas as pd
from helper import *

from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
from keras.callbacks import *
from keras.optimizers import *
from keras.utils import *
from keras import backend as K

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

/home/pengjun/.conda/envs/keras/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## 数据文件处理
训练数据包括12500张猫的图片和12500张狗的图片。我们为数据文件建立symbol link并划分为训练集和验证集，所使用的方法参考了[这里](https://github.com/ypwhs/dogs_vs_cats)。

In [2]:
#为数据连理symbol-link
train_data_dir, test_data_dir = prepare_data_file()

100%|██████████| 12500/12500 [00:00<00:00, 188439.61it/s]


## 基准模型
作为迁移学习的基础，这里我们使用ResNet50为基准模型：
- [ResNet](https://arxiv.org/abs/1512.03385)

在导出预训练模型特征时，我们所使用的方法参考了[这里](https://github.com/ypwhs/dogs_vs_cats)。

我们首先冻结所有Resnet的权重参数，只训练全链接层。我们在融合模型中已经导出了所有训练数据和测试数据在Resnet上的特征，基于这些特征，我们训练猫狗问题的网络。

In [3]:
#导入训练数据和测试数据
X_train, Y_train, X_test = load_feature_data("feature_resnet50.h5")

In [ ]:
#构造模型并显示所有网络层的名称
input_tensor = Input(X_train.shape[1:])
x = Dropout(0.5)(input_tensor)
x = Dense(1, activation='sigmoid')(x)
model = Model(input_tensor, x)

model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [7]:
#训练模型并导出权重参数
filepath="resnet50-tune0-best_weight.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min',save_weights_only=True)
callbacks_list = [checkpoint]
model.fit(X_train, Y_train, batch_size=128, epochs=20, validation_split=0.2, shuffle=True,
         callbacks=callbacks_list)

Train on 20000 samples, validate on 5000 samples
Epoch 1/20
20000/20000 [==============================] - 0s 13us/step - loss: 0.0663 - acc: 0.9743 - val_loss: 0.0564 - val_acc: 0.9784

Epoch 00001: val_loss improved from inf to 0.05638, saving model to resnet50-tune0-best_weight.h5
Epoch 2/20
20000/20000 [==============================] - 0s 15us/step - loss: 0.0691 - acc: 0.9743 - val_loss: 0.0562 - val_acc: 0.9790

Epoch 00002: val_loss improved from 0.05638 to 0.05624, saving model to resnet50-tune0-best_weight.h5
Epoch 3/20
20000/20000 [==============================] - 0s 14us/step - loss: 0.0673 - acc: 0.9748 - val_loss: 0.0592 - val_acc: 0.9798

Epoch 00003: val_loss did not improve
Epoch 4/20
20000/20000 [==============================] - 0s 15us/step - loss: 0.0648 - acc: 0.9754 - val_loss: 0.0569 - val_acc: 0.9802

Epoch 00004: val_loss did not improve
Epoch 5/20
20000/20000 [==============================] - 0s 20us/step - loss: 0.0655 - acc: 0.9748 - val_loss: 0.0633 - va

In [8]:
#在测试集上进行预测并导出预测值
predict_on_model(test_data_dir, X_test, model, "pred-resnet50-tune0.csv")

12500/12500 [==============================] - 0s 32us/step
Found 12500 images belonging to 1 classes.


/home/pengjun/DLND/Cat_vs_Dog/helper.py:115: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  df.set_value(index-1, 'label', y_test[i])


这一模型在Kaggle上的得分为0.07529，还不错。下一步我们将开始Fine-tuning基于Resnet50的猫狗分类模型。

## Fine Tuning
我们将放开Resnet50中的一些单元的权值，让它们是可学习的，以此训练我们的猫狗分类网络。

### Fine Tune 0
这里，我们首先在冻结全部权重情况下对全链接层进行训练。我们引入数据增强以获得更为泛化的数据集。

In [3]:
#构造模型
x_input = Input((224, 224, 3))
x_input = Lambda(xception.preprocess_input)(x_input)

base_model = ResNet50(input_tensor=x_input, weights='imagenet', include_top=False, pooling = 'avg')

for layer in base_model.layers:
    layer.trainable = False
    
x = Dropout(0.5)(base_model.output)
x = Dense(1, activation='sigmoid')(x)
model = Model(base_model.input, x)
model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])

for i in range(len(model.layers)):
    print(i,model.layers[i].name)

0 input_1
1 conv1_pad
2 conv1
3 bn_conv1
4 activation_1
5 max_pooling2d_1
6 res2a_branch2a
7 bn2a_branch2a
8 activation_2
9 res2a_branch2b
10 bn2a_branch2b
11 activation_3
12 res2a_branch2c
13 res2a_branch1
14 bn2a_branch2c
15 bn2a_branch1
16 add_1
17 activation_4
18 res2b_branch2a
19 bn2b_branch2a
20 activation_5
21 res2b_branch2b
22 bn2b_branch2b
23 activation_6
24 res2b_branch2c
25 bn2b_branch2c
26 add_2
27 activation_7
28 res2c_branch2a
29 bn2c_branch2a
30 activation_8
31 res2c_branch2b
32 bn2c_branch2b
33 activation_9
34 res2c_branch2c
35 bn2c_branch2c
36 add_3
37 activation_10
38 res3a_branch2a
39 bn3a_branch2a
40 activation_11
41 res3a_branch2b
42 bn3a_branch2b
43 activation_12
44 res3a_branch2c
45 res3a_branch1
46 bn3a_branch2c
47 bn3a_branch1
48 add_4
49 activation_13
50 res3b_branch2a
51 bn3b_branch2a
52 activation_14
53 res3b_branch2b
54 bn3b_branch2b
55 activation_15
56 res3b_branch2c
57 bn3b_branch2c
58 add_5
59 activation_16
60 res3c_branch2a
61 bn3c_branch2a
62 activatio

In [21]:
#训练模型并保存在验证集上损失函数最小的权重
gen = ImageDataGenerator(validation_split=0.2)
train_generator = gen.flow_from_directory(train_data_dir, (224, 224), shuffle=True, 
                                          batch_size=32,class_mode='binary')
filepath="resnet50-best_weight_freeze.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min',save_weights_only=True)
callbacks_list = [checkpoint]

model.fit_generator(
        train_generator,
        steps_per_epoch=625,
        epochs=10,
        validation_data=train_generator,
        validation_steps=150,
        callbacks = callbacks_list)

Found 25000 images belonging to 2 classes.
Epoch 1/10
625/625 [==============================] - 119s 191ms/step - loss: 0.0956 - acc: 0.9650 - val_loss: 0.1223 - val_acc: 0.9602

Epoch 00001: val_loss improved from inf to 0.12233, saving model to resnet50-best_weight_freeze.h5
Epoch 2/10
625/625 [==============================] - 119s 190ms/step - loss: 0.0902 - acc: 0.9661 - val_loss: 0.1136 - val_acc: 0.9623

Epoch 00002: val_loss improved from 0.12233 to 0.11362, saving model to resnet50-best_weight_freeze.h5
Epoch 3/10
625/625 [==============================] - 119s 191ms/step - loss: 0.0894 - acc: 0.9639 - val_loss: 0.1168 - val_acc: 0.9629

Epoch 00003: val_loss did not improve
Epoch 4/10
625/625 [==============================] - 119s 191ms/step - loss: 0.0901 - acc: 0.9665 - val_loss: 0.1169 - val_acc: 0.9658

Epoch 00004: val_loss did not improve
Epoch 5/10
625/625 [==============================] - 120s 192ms/step - loss: 0.0945 - acc: 0.9644 - val_loss: 0.1215 - val_acc: 0.

In [4]:
predict_on_resnet(12500, 224, 224, test_data_dir, model, "resnet50-best_weight_freeze.h5", "pred-resnet50-freeze.csv")

100%|██████████| 12500/12500 [00:39<00:00, 317.76it/s]


12500/12500 [==============================] - 65s 5ms/step


  0%|          | 0/12500 [00:00<?, ?it/s]/home/pengjun/DLND/Cat_vs_Dog/helper.py:133: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  df.set_value(i, 'label', y_test[i])
100%|██████████| 12500/12500 [00:00<00:00, 257512.64it/s]


这个模型在kaggle的得分是0.12619。

### Fine-tuning 1
开放162层以上的权重优化

In [5]:
for layer in model.layers:
    layer.trainable = False
    
for layer in model.layers[162:]:
    layer.trainable = True
    
model.load_weights('resnet50-best_weight_freeze.h5')
model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [6]:
#训练模型并保存在验证集上损失函数最小的权重
gen = ImageDataGenerator(rotation_range=40,
                        width_shift_range=0.2,
                        height_shift_range=0.2,
                        shear_range=0.2,
                        zoom_range=0.2,
                        horizontal_flip=True,
                        validation_split=0.2)
train_generator = gen.flow_from_directory(train_data_dir, (224, 224), shuffle=True, 
                                          batch_size=32,class_mode='binary')

filepath="resnet50-best_weight_fine_tuning-1.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min',save_weights_only=True)
callbacks_list = [checkpoint]

model.fit_generator(
        train_generator,
        steps_per_epoch=625,
        epochs=5,
        validation_data=train_generator,
        validation_steps=150,
        callbacks = callbacks_list)

Found 25000 images belonging to 2 classes.
Epoch 1/5
625/625 [==============================] - 125s 200ms/step - loss: 0.0766 - acc: 0.9715 - val_loss: 0.0486 - val_acc: 0.9823

Epoch 00001: val_loss improved from inf to 0.04864, saving model to resnet50-best_weight_fine_tuning-1.h5
Epoch 2/5
625/625 [==============================] - 124s 198ms/step - loss: 0.0528 - acc: 0.9808 - val_loss: 0.1321 - val_acc: 0.9615

Epoch 00002: val_loss did not improve
Epoch 3/5
625/625 [==============================] - 124s 198ms/step - loss: 0.0428 - acc: 0.9833 - val_loss: 0.0467 - val_acc: 0.9838

Epoch 00003: val_loss improved from 0.04864 to 0.04666, saving model to resnet50-best_weight_fine_tuning-1.h5
Epoch 4/5
625/625 [==============================] - 124s 198ms/step - loss: 0.0325 - acc: 0.9877 - val_loss: 0.0727 - val_acc: 0.9844

Epoch 00004: val_loss did not improve
Epoch 5/5
625/625 [==============================] - 124s 198ms/step - loss: 0.0274 - acc: 0.9906 - val_loss: 0.0357 - va

In [7]:
predict_on_resnet(12500, 224, 224, test_data_dir, model, "resnet50-best_weight_fine_tuning-1.h5", "pred-resnet50-fine_tuning-1.csv")

100%|██████████| 12500/12500 [00:38<00:00, 323.02it/s]


12500/12500 [==============================] - 64s 5ms/step


  0%|          | 0/12500 [00:00<?, ?it/s]/home/pengjun/DLND/Cat_vs_Dog/helper.py:133: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  df.set_value(i, 'label', y_test[i])
100%|██████████| 12500/12500 [00:00<00:00, 257554.38it/s]


这一轮Tuning的最终得分为：0.07575。

### Fine-tuning 2
开放152层以上的权重优化

In [8]:
for layer in model.layers:
    layer.trainable = False
    
for layer in model.layers[152:]:
    layer.trainable = True
    
model.load_weights('resnet50-best_weight_fine_tuning-1.h5')
model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [9]:
gen = ImageDataGenerator(rotation_range=40,
                        width_shift_range=0.2,
                        height_shift_range=0.2,
                        shear_range=0.2,
                        zoom_range=0.2,
                        horizontal_flip=True,
                        validation_split=0.2)
train_generator = gen.flow_from_directory(train_data_dir, (224, 224), shuffle=True, 
                                          batch_size=32,class_mode='binary')

filepath="resnet50-best_weight_fine_tuning-2.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min',save_weights_only=True)
callbacks_list = [checkpoint]

model.fit_generator(
        train_generator,
        steps_per_epoch=625,
        epochs=5,
        validation_data=train_generator,
        validation_steps=150,
        callbacks = callbacks_list)

Found 25000 images belonging to 2 classes.
Epoch 1/5
625/625 [==============================] - 133s 212ms/step - loss: 0.0362 - acc: 0.9885 - val_loss: 0.0357 - val_acc: 0.9867

Epoch 00001: val_loss improved from inf to 0.03565, saving model to resnet50-best_weight_fine_tuning-2.h5
Epoch 2/5
625/625 [==============================] - 132s 211ms/step - loss: 0.0191 - acc: 0.9935 - val_loss: 0.0275 - val_acc: 0.9898

Epoch 00002: val_loss improved from 0.03565 to 0.02745, saving model to resnet50-best_weight_fine_tuning-2.h5
Epoch 3/5
625/625 [==============================] - 132s 211ms/step - loss: 0.0164 - acc: 0.9944 - val_loss: 0.0538 - val_acc: 0.9840

Epoch 00003: val_loss did not improve
Epoch 4/5
625/625 [==============================] - 132s 211ms/step - loss: 0.0096 - acc: 0.9967 - val_loss: 0.0277 - val_acc: 0.9915

Epoch 00004: val_loss did not improve
Epoch 5/5
625/625 [==============================] - 132s 211ms/step - loss: 0.0075 - acc: 0.9975 - val_loss: 0.0651 - va

In [10]:
predict_on_resnet(12500, 224, 224, test_data_dir, model, "resnet50-best_weight_fine_tuning-2.h5", "pred-resnet50-fine_tuning-2.csv")

100%|██████████| 12500/12500 [00:38<00:00, 322.56it/s]


12500/12500 [==============================] - 64s 5ms/step


  0%|          | 0/12500 [00:00<?, ?it/s]/home/pengjun/DLND/Cat_vs_Dog/helper.py:133: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  df.set_value(i, 'label', y_test[i])
100%|██████████| 12500/12500 [00:00<00:00, 254873.73it/s]


从loss上看，这个模型已经有点过拟合了，不过没关系，我们只是把它所谓增强模型的初始值。这一轮Tuning的最终得分为：0.07564。

### Fine-tuning 3
开放140层以上的权重优化。同时，我们对图片生成器进行增强。

In [11]:
for layer in model.layers:
    layer.trainable = False
    
for layer in model.layers[140:]:
    layer.trainable = True
    
model.load_weights('resnet50-best_weight_fine_tuning-2.h5')
model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [12]:
gen = ImageDataGenerator(rotation_range=40,
                        width_shift_range=0.2,
                        height_shift_range=0.2,
                        shear_range=0.2,
                        zoom_range=0.2,
                        horizontal_flip=True,
                        validation_split=0.2)

train_generator = gen.flow_from_directory(train_data_dir, (224, 224), shuffle=True, 
                                          batch_size=64,class_mode='binary')
filepath="resnet50-best_weight_fine_tuning-3.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min',save_weights_only=True)
callbacks_list = [checkpoint]

model.fit_generator(
        train_generator,
        steps_per_epoch=1250,
        epochs=10,
        validation_data=train_generator,
        validation_steps=300,
        callbacks = callbacks_list)

Found 25000 images belonging to 2 classes.
Epoch 1/10
1250/1250 [==============================] - 703s 563ms/step - loss: 0.0619 - acc: 0.9765 - val_loss: 0.0883 - val_acc: 0.9678

Epoch 00001: val_loss improved from inf to 0.08826, saving model to resnet50-best_weight_fine_tuning-3.h5
Epoch 2/10
1250/1250 [==============================] - 695s 556ms/step - loss: 0.0352 - acc: 0.9866 - val_loss: 0.0586 - val_acc: 0.9797

Epoch 00002: val_loss improved from 0.08826 to 0.05857, saving model to resnet50-best_weight_fine_tuning-3.h5
Epoch 3/10
1250/1250 [==============================] - 695s 556ms/step - loss: 0.0243 - acc: 0.9912 - val_loss: 0.0510 - val_acc: 0.9814

Epoch 00003: val_loss improved from 0.05857 to 0.05105, saving model to resnet50-best_weight_fine_tuning-3.h5
Epoch 4/10
1250/1250 [==============================] - 695s 556ms/step - loss: 0.0198 - acc: 0.9927 - val_loss: 0.0561 - val_acc: 0.9793

Epoch 00004: val_loss did not improve
Epoch 5/10
1250/1250 [===============

In [13]:
predict_on_resnet(12500, 224, 224, test_data_dir, model, "resnet50-best_weight_fine_tuning-3.h5", "pred-resnet50-fine_tuning-3.csv")

100%|██████████| 12500/12500 [00:38<00:00, 322.97it/s]


12500/12500 [==============================] - 64s 5ms/step


  0%|          | 0/12500 [00:00<?, ?it/s]/home/pengjun/DLND/Cat_vs_Dog/helper.py:133: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  df.set_value(i, 'label', y_test[i])
100%|██████████| 12500/12500 [00:00<00:00, 255382.74it/s]


最终，Fine tuning的得分为：0.07259。

## 保存效果最好的那个模型
从最终的得分看出，目前效果最好的模型是Fine-tuning-2。我们载入全部模型，并将基础模型部分的权重分开保存起来。

In [ ]:
base_model = ResNet50(input_tensor=Input((224, 224, 3)), weights='imagenet', include_top=False, pooling = 'avg')

for layer in base_model.layers:
    layer.trainable = False
    
x = Dropout(0.5)(base_model.output)
x = Dense(1, activation='sigmoid')(x)
model = Model(base_model.input, x)
model.load_weights('resnet50-best_weight_fine_tuning-3.h5')

base_model.save_weights("fine_tuned_resnet50.h5")